<a href="https://colab.research.google.com/github/NUMAIRn/NUMAIRn/blob/main/Fine-tuning%20bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 24.7 MB/s eta 0:00:00


In [1]:
!pip install -q transformers datasets

In [27]:
import pandas as pd
from datasets import Dataset
df = pd.read_csv("/content/FinalDataset.csv", encoding = 'unicode_escape')
dfv = pd.read_csv("/content/validation.csv", encoding = 'unicode_escape')
dataset = Dataset.from_pandas(df)
validation = Dataset.from_pandas(dfv)

In [7]:
dataset

Dataset({
    features: ['Project Abstracts', 'SDG1', 'SDG2', 'SDG3', 'SDG4', 'SDG5', 'SDG6', 'SDG7', 'SDG8', 'SDG9', 'SDG10', 'SDG11', 'SDG12', 'SDG13', 'SDG14', 'SDG15', 'SDG16', 'SDG17'],
    num_rows: 3188
})

In [28]:
validation

Dataset({
    features: ['Project Abstracts', 'SDG1', 'SDG2', 'SDG3', 'SDG4', 'SDG5', 'SDG6', 'SDG7', 'SDG8', 'SDG9', 'SDG10', 'SDG11', 'SDG12', 'SDG13', 'SDG14', 'SDG15', 'SDG16', 'SDG17'],
    num_rows: 414
})

In [8]:
labels = [label for label in dataset.features.keys() if label not in ['Project Abstracts']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['SDG1',
 'SDG2',
 'SDG3',
 'SDG4',
 'SDG5',
 'SDG6',
 'SDG7',
 'SDG8',
 'SDG9',
 'SDG10',
 'SDG11',
 'SDG12',
 'SDG13',
 'SDG14',
 'SDG15',
 'SDG16',
 'SDG17']

In [9]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["Project Abstracts"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()

  return encoding

In [10]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset.column_names)


Map:   0%|          | 0/3188 [00:00<?, ? examples/s]

In [29]:
encoded_val = validation.map(preprocess_data, batched=True, remove_columns=validation.column_names)

Map:   0%|          | 0/414 [00:00<?, ? examples/s]

In [11]:
example = encoded_dataset[0]
print(example.keys())


dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [12]:
tokenizer.decode(example['input_ids'])

'[CLS] the objective of this project is to develop a suite of mobile and web applications that shall serve as a platform for food vendors to connect with their customers and enhance their reach. it shall let food critics review different vendors as well. it shall guide food lovers especially tourists about the best eating spots in a city. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [13]:
example['labels']

[0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [14]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['SDG2', 'SDG8', 'SDG11']

In [15]:
encoded_dataset.set_format("torch")

In [30]:
encoded_val.set_format("torch")

In [16]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
batch_size = 8
metric_name = "f1"

In [18]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

In [19]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

In [20]:
encoded_dataset[0]['labels'].type()

'torch.FloatTensor'

In [31]:
encoded_val[0]['labels'].type()

'torch.FloatTensor'

In [21]:
encoded_dataset['input_ids'][0]

tensor([  101,  1996,  7863,  1997,  2023,  2622,  2003,  2000,  4503,  1037,
         7621,  1997,  4684,  1998,  4773,  5097,  2008,  4618,  3710,  2004,
         1037,  4132,  2005,  2833, 17088,  2000,  7532,  2007,  2037,  6304,
         1998, 11598,  2037,  3362,  1012,  2009,  4618,  2292,  2833,  4401,
         3319,  2367, 17088,  2004,  2092,  1012,  2009,  4618,  5009,  2833,
        10205,  2926,  9045,  2055,  1996,  2190,  5983,  7516,  1999,  1037,
         2103,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [22]:
#forward pass
outputs = model(input_ids=encoded_dataset['input_ids'][0].unsqueeze(0), labels=encoded_dataset[0]['labels'].unsqueeze(0))
outputs

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


SequenceClassifierOutput(loss=tensor(0.7229, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 0.1324, -0.1660, -0.1566, -0.3985, -0.5115,  0.3377,  0.1793, -0.2526,
          0.1511, -0.1318, -0.1840,  0.4330, -0.4036, -0.2785, -0.1316,  0.0565,
          0.7053]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [32]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset,
    eval_dataset=encoded_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [33]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.304321,0.495614,0.668732,0.031401
2,0.280500,0.272818,0.633710,0.748407,0.072464
3,0.155600,0.274813,0.625767,0.744057,0.089372
4,0.105900,0.285315,0.637817,0.750625,0.115942
5,0.105900,0.278886,0.652708,0.763118,0.125604


TrainOutput(global_step=1995, training_loss=0.15766175922594572, metrics={'train_runtime': 489.0981, 'train_samples_per_second': 32.591, 'train_steps_per_second': 4.079, 'total_flos': 1048638766156800.0, 'train_loss': 0.15766175922594572, 'epoch': 5.0})

In [58]:
text = "As people are becoming health conscious nowadays, a significant increase can be seen among the members at various Fitness & Health Clubs. However, it is difficult for the fitness club owners to manually manage the admission entries, keep data of new & old members, scheduling events, etc. The android app project proposed here aims at automating the management system for Fitness & Training Health clubs. New members can easily get registered with the fitness club by selecting the desired membership program & completing the payment procedure. The members can then access the app by using login credentials. The users can select a goal for themselves & set a target. The app will notify the users of the upcoming events for which they can get themselves registered if interested. The app consists of various diet plans from which the members can choose as per their body weight & type. This application will guide the members on various body exercises through videos. Also, the users can attend the fitness club’s gym. Additionally, a motivational video section is also included in this app to boost the members' morale & keep them motivated. The members can add or edit notes. The club management team can view the list of all the members & can filter them as per their membership program. The club owners can add new events for the members. Thus, this Android App will provide a user-friendly experience to both the members & the fitness club owners."

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

In [59]:
logits = outputs.logits
logits.shape

torch.Size([1, 17])

In [60]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

['SDG3', 'SDG9']


In [40]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [41]:
model_save_name = 'BertclassifierF1.pt'
path = F"/content/gdrive/My Drive/{model_save_name}"
torch.save(model.state_dict(), path)

In [42]:
!ls /content/gdrive/MyDrive

 BertclassifierF1.pt   Bertclassifier.pt  'Colab Notebooks'


In [ ]:
model_save_name = 'BertclassifierF1.pt'
path = F"/content/gdrive/My Drive/{model_save_name}"
model.load_state_dict(torch.load(path))

NameError: ignored